In [7]:
import pandas as pd
import numpy as np
import datetime as dt

In [37]:
###### - Table 1. daily_summary - ######
##### this is the main ouput table needed, should have around 700 rows (20 participants*5days*7weeks) ##
##### each row represents a day for a single participant

## duration represented in seconds ## 

daily_summary = [] #len should be roughly  700 (20ppl*5days*7weeks) 
daily_summary = pd.DataFrame({'pID': '01', #str. '01' - '20' 
                       'app_version':0,#0 = baseline app; 1 = intervention app 
                       'date': dt.datetime(2017, 9, 4),
                       'day_week':0, # Integer (0 - 6 Mon-Sun) corresponding to the day of the week
                        'day_start':dt.time(8,45),#the moment since which data come in, as a datetime.time objects
                        'day_end':dt.time(16,45), #the moment after which no more data come in 
                        'timely_breaks':3, #integer
                        'summed_active':3600, 
                        'prolonged_sitting':3600*2,
                        'healthy_sitting':3600*4,
                        'lost_period': 1800, #period when user opted in tracking but no data is recorded due to disconnection or unresponsive board   
                        'paused_tracking':1800 # period when user paused tracking  
                       }, index = [0])

#re-ordering the column
daily_summary= daily_summary[['app_version','day_week','pID','date','day_start','day_end','timely_breaks','prolonged_sitting','healthy_sitting','summed_active','lost_period','paused_tracking']]

daily_summary

,app_version,day_week,pID,date,day_start,day_end,timely_breaks,prolonged_sitting,healthy_sitting,summed_active,lost_period,paused_tracking
0,0,0,01,2017-09-04,08:45:00,16:45:00,3,7200,14400,3600,1800,1800


In [16]:
###### -Table 2: event_record - ######  
##### break-down of Table 1, each row is an event, or the end of an event(transitional point)##

event_record = pd.DataFrame({'pID': '01', #str. '01' - '20' 
                        'app_version':1,#0 = baseline app; 1 = intervention app 
                        'datetime': dt.datetime(2017,9,5,13,3) ,
                        'day_week':0, # Integer (0 - 6 Mon-Sun) corresponding to the day of the week
                        'transition_type': 1,#integer: 1 = transition from break; 
                                            #0 = transition from sit; -1 = transition from unresponsive board/paused tracking  
                        'duration':55.25*60,# duration of the episode proceeding this transition in seconds
                        'reminder1':45*60,'reminder2':60*60,'reminder3':75*60, #read from settings
                        'snooze':2, #how many times of snooze recorded since last transition 
                        'goal1':3,'goal2':61*60},index = [0])

event_record = event_record [['app_version','day_week','pID','datetime','transition_type','duration','reminder1','reminder2','reminder3','snooze','goal1','goal2']]

event_record


,app_version,day_week,pID,datetime,transition_type,duration,reminder1,reminder2,reminder3,snooze,goal1,goal2
0,1,0,01,2017-09-05 13:03:00,1,3315.0,2700,3600,4500,2,3,3660


In [34]:
##### - Table 3: count_record - #####
# break-down of events; each row is an epoch
# datetime (end of the current event) can be used for grouping epochs and linking with Table 2 ## 

count_record = pd.DataFrame({'day_week':[0,0,0,0], 
            'app_version':[1,1,1,1],
            'pID': ['01','01','01','01'],
            'epoch_end':[dt.datetime(2017, 9, 4,14,5,0),dt.datetime(2017, 9, 4,14,5,15),dt.datetime(2017, 9, 4,14,5,30),dt.datetime(2017, 9, 4,14,5,45)],
            'datetime':[dt.datetime(2017, 9, 4,14,8,0),dt.datetime(2017, 9, 4,14,8,0),dt.datetime(2017, 9, 4,14,8,0),dt.datetime(2017, 9, 4,14,8,0)],
            'cpe_cup':[2,15,13,float('nan')],#count per epoch 
            'cpe_wrist':[8,11,0,float('nan')],
            'event_label':[0,1,1,-1]#1 for active, 0 for inactive, -1 for no tracking data 
               })
count_record

,app_version,cpe_cup,cpe_wrist,datetime,day_week,epoch_end,event_label,pID
0,1,2.0,8.0,2017-09-04 14:08:00,0,2017-09-04 14:05:00,0,01
1,1,15.0,11.0,2017-09-04 14:08:00,0,2017-09-04 14:05:15,1,01
2,1,13.0,0.0,2017-09-04 14:08:00,0,2017-09-04 14:05:30,1,01
3,1,NaN,NaN,2017-09-04 14:08:00,0,2017-09-04 14:05:45,-1,01


In [10]:
###### - Table 3: break_details - ######
### this can be seen as a subset of Table 2 delineated with count records ## 
## each row is a break with the pID and datetime (end of break) as keys for linking with Table 2 - ###### 

break_record = []
break_record.append({'day_week':0, 
                    'app_version':1,
                    'pID': '01',
                    'datetime': dt.datetime(2017, 9, 4,14,7), #the end of the break, to be consistent with table 2 
                    'counts_wrist':[8,11,25,2,25,6,1,6],#counts per epoch, the len also indicates the duration 
                    'counts_cup':[2,15,20,0,14,12,0,7],#count_records of the cup in the same timeframe (eg. x epochs backward from 'datetime') 
                    })

break_df = pd.DataFrame(break_record)
break_df = break_df[['app_version','day_week','pID','datetime','counts_wrist','counts_cup']] 
break_df

,app_version,day_week,pID,datetime,counts_wrist,counts_cup
0,1,0,01,2017-09-04 14:07:00,"[8, 11, 25, 2, 25, 6, 1, 6]","[2, 15, 20, 0, 14, 12, 0, 7]"


In [11]:
### -- Table 4: connection status -- ###
## you can redesign the structure or follow the web database 
connection_status = pd.DataFrame({'day_week':0, 
                        'app_version':1,
                        'pID': '01',
                        'datetime': dt.datetime(2017, 9, 4,14,7),
                        'device_type': 1, # 1 for wrist? 2 for wrist?
                        'connect': True # False for 'going from connected to disconnect' 
                                 },index = [0])

connection_status

,app_version,connect,datetime,day_week,device_type,pID
0,1,True,2017-09-04 14:07:00,0,1,01


## append new data 

In [12]:
# append new data to sit_break_transitions 
sit_break_transistions = sit_break_transistions.append ({'pID': '01', 
                        'app_version':1,
                        'datetime': dt.datetime(2017, 9, 4,14,7), 
                        'day_week':0,
                        'transition_type': '0',
                        'duration':(dt.datetime(2017, 9, 4,14,7)-dt.datetime(2017, 9, 4,14,3)).seconds, 
                        'reminder1':45*60,'reminder2':60*60,'reminder3':75*60,
                        'snooze':0,
                        'goal1':3,'goal2':61*60},ignore_index = True)
sit_break_transistions

,app_version,day_week,pID,datetime,transition_type,duration,reminder1,reminder2,reminder3,snooze,goal1,goal2
0,1,0,01,2017-09-05 13:03:00,1,3315.0,2700,3600,4500,2,3,3660
1,1,0,01,2017-09-04 14:07:00,0,240.0,2700,3600,4500,0,3,3660
